In [74]:


#imports
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split

import os
import json
from pandas import json_normalize

def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df


In [75]:
train = load_df(nrows=100000, csv_path="data/train.csv")
test = load_df("data/test.csv", 100000)
pd.set_option('display.max_columns', None)

Loaded train.csv. Shape: (100000, 55)
Loaded test.csv. Shape: (100000, 53)


In [76]:
#df = df.loc[:, (df.isnull().sum(axis=0) <= max_number_of_nas)]
#train = train.loc[:, (train.eq("not available in demo dataset"))]

# Find the columns where values are not available
useless_cols = [col for col in train.columns if train[col].eq("not available in demo dataset").all()]
# Drop these columns from the dataframe
train.drop(useless_cols,
        axis=1,
        inplace=True)
train.info()

# Find the columns where values are not available
useless_cols = [col for col in train.columns if train[col].isna().all()]
# Drop these columns from the dataframe
train.drop(useless_cols,
        axis=1,
        inplace=True)
train.info()

C:\Users\steff\anaconda3\lib\site-packages\pandas\core\ops\array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 38 columns):
 #   Column                                        Non-Null Count   Dtype 
---  ------                                        --------------   ----- 
 0   channelGrouping                               100000 non-null  object
 1   date                                          100000 non-null  int64 
 2   fullVisitorId                                 100000 non-null  object
 3   sessionId                                     100000 non-null  object
 4   socialEngagementType                          100000 non-null  object
 5   visitId                                       100000 non-null  int64 
 6   visitNumber                                   100000 non-null  int64 
 7   visitStartTime                                100000 non-null  int64 
 8   device.browser                                100000 non-null  object
 9   device.operatingSystem                        100000 non-nul

In [77]:
const_cols = [c for c in train.columns if train[c].nunique(dropna=False)==1 ]
const_cols

['socialEngagementType', 'totals.visits']

In [78]:
print("Variables not in test but in train : ", set(train.columns).difference(set(test.columns)))

Variables not in test but in train :  {'trafficSource.campaignCode', 'totals.transactionRevenue'}


In [79]:
cols_to_drop = const_cols + ['sessionId']

train_df = train.drop(cols_to_drop + ["trafficSource.campaignCode"], axis=1)
test_df = test.drop(cols_to_drop, axis=1)

In [88]:
from sklearn import model_selection, preprocessing, metrics
import datetime

# Impute 0 for missing target values
train_df["totals.transactionRevenue"].fillna(0.0, inplace=True)
train_df["totals.pageviews"].fillna(0.0, inplace=True)
train_df["totals.bounces"].fillna(0.0, inplace=True)
train_df["totals.newVisits"].fillna(0.0, inplace=True)
train_y = train_df["totals.transactionRevenue"].values
train_id = train_df["fullVisitorId"].values
test_id = test_df["fullVisitorId"].values


# label encode the categorical variables and convert the numerical variables to float
cat_cols = ["channelGrouping", "device.browser", 
            "device.deviceCategory", "device.operatingSystem", 
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')) + list(test_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    test_df[col] = lbl.transform(list(test_df[col].values.astype('str')))

train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype(float)

num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
    test_df[col] = test_df[col].astype(float)

# Split the train dataset into development and valid based on time 
#train_df["totals.transactionRevenue"] = train_df["totals.transactionRevenue"].astype('float')
dev_df = train_df[train_df['date']<=20170531]
val_df = train_df[train_df['date']>2017531]
dev_y = np.log1p(dev_df["totals.transactionRevenue"].values)
val_y = np.log1p(val_df["totals.transactionRevenue"].values)

dev_X = dev_df[cat_cols + num_cols] 
val_X = val_df[cat_cols + num_cols] 
test_X = test_df[cat_cols + num_cols]

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [89]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    dev_X, dev_y, random_state=0)

In [97]:
X_train.head()

,channelGrouping,device.browser,device.deviceCategory,device.operatingSystem,geoNetwork.city,geoNetwork.continent,geoNetwork.country,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,trafficSource.adContent,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.isTrueDirect,totals.hits,totals.pageviews,visitNumber,visitStartTime,totals.bounces,totals.newVisits
85573,1,28,0,13,264,2,143,72,363,71,9,31,3,7395,6,3,4,492,2,1118,99,1,1,1.0,1.0,1.0,1.480437e+09,1.0,1.0
35822,2,28,0,13,70,4,155,0,1073,127,20,31,3,7395,6,3,8,492,0,1118,0,1,0,2.0,2.0,1.0,1.493707e+09,0.0,1.0
80607,2,28,1,1,0,2,49,0,6182,0,13,31,3,7395,6,3,8,492,0,1118,0,1,0,5.0,5.0,1.0,1.471878e+09,0.0,1.0
875,4,28,0,17,264,2,143,72,0,71,9,31,3,7395,6,3,8,318,5,1118,10,1,1,18.0,12.0,1.0,1.472838e+09,0.0,1.0
87560,2,28,1,1,0,2,143,0,7337,0,9,31,3,7395,6,3,8,492,0,1118,0,1,0,1.0,1.0,1.0,1.489842e+09,1.0,1.0


In [94]:
reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train, y_train)
val_pred = reg.predict(X_test[1:2])

In [95]:
reg.score(X_test, y_test)

0.3322613937319503

In [96]:
from sklearn import metrics
pred_val[pred_val<0] = 0
val_pred_df = pd.DataFrame({"fullVisitorId":val_df["fullVisitorId"].values})
val_pred_df["transactionRevenue"] = val_df["totals.transactionRevenue"].values
val_pred_df["PredictedRevenue"] = np.expm1(pred_val)
#print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))
val_pred_df = val_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print(np.sqrt(metrics.mean_squared_error(np.log1p(val_pred_df["transactionRevenue"].values), np.log1p(val_pred_df["PredictedRevenue"].values))))

NameError: name 'pred_val' is not defined

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

some_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="constant", fill_value=0)),
    ('GBR', GradientBoostingRegressor(random_state=0))
])

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

train_df["revenue_cat"] = pd.cut(train_df["totals.transactionRevenue"],
                               bins=[0., 1.5, 3.0, 4.5, 6., np.inf],
                               labels=[1, 2, 3, 4, 5])

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(train_df, train_df["revenue_cat"]):
    strat_train_set = train_df.loc[train_index]
    strat_test_set = train_df.loc[test_index]
    

#for set_ in (strat_train_set, strat_test_set):
 #   set_.drop("income_cat", axis=1, inplace=True)

strat_train_set = strat_train_set.drop(['revenue_cat'], axis=1)
strat_test_set= strat_test_set.drop(['revenue_cat'], axis=1)



We can tell that there appears to be no missing values

In [ ]:

import matplotlib.pyplot as plt

train.plot(kind="scatter", x="geoNetwork.longitude", y="geoNetwork.latitude", alpha=0.4,
    #s=training["totals.transactionRevenut"]/100, label="transactionRevenue", 
    figsize=(10,7),
    c="totals.transactionRevenue", cmap=plt.get_cmap("jet"), colorbar=True,
    sharex=False)


In [ ]:
train["geoNetwork.longitude"]

In [ ]:
train["totals.transactionRevenue"]

In [ ]:
train.head()

Maybe add columns for lat/long using another dataset like https://simplemaps.com/data/world-cities

In [ ]:
payingCustomers = train.loc[train['totals.transactionRevenue'].notna()]
payingCustomers

In [ ]:
payingCustomers.describe()

In [ ]:
payingCustomers.info()

In [ ]:
payingCustomers["channelGrouping"].value_counts()

In [ ]:
payingCustomers["date"].value_counts()

In [ ]:
import seaborn as sns
cor = payingCustomers.corr()
plt.figure(figsize=(3,3))
sns.heatmap(cor, annot=True)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

some_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="constant", fill_value=0))
])

TODO:

Impute missing values
    Set revnue to 0 if not present
scale

 0   channelGrouping                               100000 non-null  object
* 1   date                                          100000 non-null  int64 
- 2   fullVisitorId                                 100000 non-null  object
- 3   sessionId                                     100000 non-null  object
 4   socialEngagementType                          100000 non-null  object
- 5   visitId                                       100000 non-null  int64 
 6   visitNumber                                   100000 non-null  int64 
* 7   visitStartTime                                100000 non-null  int64 
 8   device.browser                                100000 non-null  object
 9   device.operatingSystem                        100000 non-null  object
 10  device.isMobile                               100000 non-null  bool  
 11  device.deviceCategory                         100000 non-null  object
 12  geoNetwork.continent                          100000 non-null  object
 13  geoNetwork.subContinent                       100000 non-null  object
 14  geoNetwork.country                            100000 non-null  object
 15  geoNetwork.region                             100000 non-null  object
 16  geoNetwork.metro                              100000 non-null  object
 17  geoNetwork.city                               100000 non-null  object
* 18  geoNetwork.networkDomain                      100000 non-null  object
 19  totals.visits                                 100000 non-null  object
 20  totals.hits                                   100000 non-null  object
 21  totals.pageviews                              99993 non-null   object
 22  totals.bounces                                48916 non-null   object
 23  totals.newVisits                              77263 non-null   object
 24  totals.transactionRevenue                     1399 non-null    object
 25  trafficSource.campaign                        100000 non-null  object
 26  trafficSource.source                          100000 non-null  object
 27  trafficSource.medium                          100000 non-null  object
 28  trafficSource.keyword                         44218 non-null   object
 29  trafficSource.isTrueDirect                    30454 non-null   object
 30  trafficSource.referralPath                    36473 non-null   object
 31  trafficSource.adwordsClickInfo.page           2574 non-null    object
 32  trafficSource.adwordsClickInfo.slot           2574 non-null    object
 33  trafficSource.adwordsClickInfo.gclId          2625 non-null    object
 34  trafficSource.adwordsClickInfo.adNetworkType  2574 non-null    object
 35  trafficSource.adwordsClickInfo.isVideoAd      2574 non-null    object
 36  trafficSource.adContent                       1325 non-null    object
 37  trafficSource.campaignCode                    1 non-null       object

In [ ]:
train["geoNetwork.networkDomain"].head()

In [ ]:
payingCustomers["totals.transactionRevenue"].head()

In [ ]:
payingCustomers["totals.transactionRevenue"].describe()

In [ ]:
payingCustomers.head()

## Deployment

Suggestion for deployment: Write a web API which can receive data like one or more rows from the test dataset and return a prediction for that data. We are thinking the client will send the data as JSON and receive a JSON response.

In order to make this happen, we will need to have a way to transform our data to JSON format, as well as a way to transform it back to a dataframe. 

### TODO/Checklist

* Create pipeline
    * Create transformers for:
        * Remove useless columns
        * Impute missing values
        * Transform total revenue to natural log of total revenue
        
        
* Test the model to make sure it performs well-ish
        
* Store model to a file using pickle or joblib
    
    

* Create API - The API will receive raw data in the same format as our initial CSV files. It will then have to:
    * Remove missing columns
    * impute missing values
    * any other preprocessing
    * Make prediction
    * Return a JSON object containing customer ID and transactionrevenue. 
* Test API
   
We might start by only accepting single row data and then expand to allow multiple row data. 

### If we have time:
* Display data in nice ways to gain insights
* Try different models
* Deploy API

In [98]:
{ customerId: id, totals.transactionRevenue: value }

NameError: name 'customerId' is not defined